# House Price Prediction (Ames Dataset)

## Model
- Random Forest Regressor
- One-hot encoding for categorical features
- Median imputation for numeric features
- Mode imputation for categorical features

## Evaluation Metrics (Validation Set)

- **MSE:** 858,300,293.9983  
- **RMSE:** 29,296.76  
- **Log RMSE:** 0.1553  

Lower Log RMSE indicates strong relative prediction accuracy.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [2]:
train_df = pd.read_csv("train.csv")

train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
X = train_df.drop("SalePrice", axis=1)
y = train_df["SalePrice"]

X = pd.get_dummies(X)   

X_train, X_val, y_train, y_val = train_test_split(X, y)

In [4]:
numeric_features = X.select_dtypes(include=[np.number])
categorical_features = X.select_dtypes(exclude=[np.number])

print("Numeric features:", numeric_features.shape)
print("Categorical features:", categorical_features.shape)

Numeric features: (1460, 37)
Categorical features: (1460, 251)


In [5]:
categorical_features = categorical_features.fillna(
    categorical_features.mode().iloc[0]
)

In [6]:
X_processed = pd.concat(
    [numeric_features, categorical_features],
    axis=1
)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_processed, y, 
    test_size=0.2, 
    random_state=42
)

In [8]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

In [9]:
model.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [10]:
score = model.score(X_val, y_val)
print("R² Score:", score)

R² Score: 0.888101200546844


In [11]:
feature_importance = pd.Series(
    model.feature_importances_,
    index=X_processed.columns
).sort_values(ascending=False)

feature_importance.head(10)

OverallQual    0.557724
GrLivArea      0.120651
2ndFlrSF       0.034721
TotalBsmtSF    0.033871
BsmtFinSF1     0.029017
1stFlrSF       0.025869
LotArea        0.016890
GarageArea     0.014919
YearBuilt      0.012262
GarageCars     0.011863
dtype: float64

In [12]:
test_df = pd.read_csv("test.csv")
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [13]:
test_numeric = test_df.select_dtypes(include=[np.number])
test_categorical = test_df.select_dtypes(exclude=[np.number])

In [14]:
test_numeric = test_numeric.fillna(test_numeric.median())
test_categorical = test_categorical.fillna(
    test_categorical.mode().iloc[0]
)

X_test = pd.concat(
    [test_numeric, test_categorical],
    axis=1
)

In [15]:
X_test = pd.get_dummies(X_test)

In [16]:
X_test = X_test.reindex(
    columns=X_processed.columns,
    fill_value=0
)

In [23]:
test_predictions = model.predict(X_test)
test_predictions[:50]

array([129484.55, 154650.25, 178934.19, 188258.5 , 206244.46, 182989.32,
       168732.88, 177251.78, 179495.03, 123093.33, 188812.41,  92099.04,
       110504.52, 151433.  , 140616.76, 381709.16, 258035.08, 317050.06,
       250567.32, 445891.02, 305814.35, 211205.36, 180501.2 , 170968.88,
       171136.77, 197228.76, 345234.45, 241849.84, 208836.75, 196993.71,
       191131.94,  98828.  , 178795.34, 296774.58, 299694.06, 218198.41,
       188996.67, 152140.04, 151300.64, 157209.68, 169659.64, 162854.5 ,
       312472.21, 230009.26, 218272.13, 192919.78, 228616.8 , 204628.  ,
       169105.5 , 152539.  ])

In [24]:
submission = pd.DataFrame({
    "Id": test_df["Id"],
    "SalePrice": test_predictions
})

submission.to_csv("submission.csv", index=False)

In [25]:
import os
os.listdir()


['.Rhistory',
 '.config',
 'Music',
 '.DS_Store',
 'unstructured program',
 '.CFUserTextEncoding',
 'Lists',
 '.zshrc',
 '.arduinoIDE',
 '.local',
 'Pictures',
 'test.csv',
 'test.cpp',
 '.zprofile',
 'Hello.class',
 'submission.csv',
 'lists.h',
 'CODE',
 'Hello.java',
 '.zsh_history',
 '.ipython',
 'Desktop',
 'Library',
 'data_description.txt',
 '.lesshst',
 'lists.cpp',
 '.cups',
 'vs codes',
 'Public',
 '.RData',
 'new practice.ipynb',
 'Movies',
 'Applications',
 '.Rapp.history',
 '.Trash',
 'train.csv',
 '.ipynb_checkpoints',
 '.jupyter',
 '.npm',
 'Documents',
 '.vscode',
 'Listsss',
 'Downloads',
 '.cache',
 '.gitconfig',
 '.zsh_sessions',
 'sample_submission.csv']

In [26]:
import pandas as pd
pd.read_csv("submission.csv").head()

,Id,SalePrice
0,1461,129484.55
1,1462,154650.25
2,1463,178934.19
3,1464,188258.50
4,1465,206244.46


In [27]:
from sklearn.metrics import mean_squared_error

# Predict on validation set
y_val_pred = model.predict(X_val)

# MSE
mse = mean_squared_error(y_val, y_val_pred)

# RMSE
rmse = np.sqrt(mse)

print("MSE:", mse)
print("RMSE:", rmse)


MSE: 858300293.9983181
RMSE: 29296.76251735536


In [28]:
rmse_log = np.sqrt(
    mean_squared_error(
        np.log1p(y_val),
        np.log1p(y_val_pred)
    )
)

print("Log RMSE:", rmse_log)


Log RMSE: 0.1552656632354743
